# Quantify transports across Iceland-Faroe Ridge by source and destination

The idea here is to use Lagrangian particle tracks to quantify model volume transports from the North Atlantic into the Arctic seas by source. This is partly motivated by an idea of Stuart's that 'all' the water heading north between Greenland-Iceland-Shetland goes through the Rockall Trough (with the northward transport west of the Rockall Bank circulating in the subpolar gyre).

I've started with the Iceland-Faroes Ridge (IFR) section. This is partly just because I have an 'emotional attachment' to the IFR having spent my first postdoc modelling it, and partly because I figured if Stuart is wrong, the transport across the IFR is likely where it will show up.

Obviously it is easier to quantify transports across sections from the model just by straight eulerian calculation, but even if this is done over the whole boundary of a closed volume it doesn't give you the pathways through the volume. A 3D streamfunction would be another possibility but that doesn't seem straightforward.

The basic idea is that each particle tracked represents a streamtube (I have no diffusion in the tracking). The volume transport along that streamtube is constant at any point along the length and determined by the transport where it crosses IFR. This we get by a simple calculation of cross-sectional area x velocity across the section.

Particles are positioned randomly in 2D along a section from Iceland to Faroes, then the same particles are tracked backwards for 2 years for sources and forwards for 6 months for destinations.

This is done using OceanParcels tracking software and Viking20x model fields on GEOMAR servers via python Jupyter notebooks. This notebook has the analysis of the tracks, the notebook running Parcels is separate. This is currently just testing the method, how many particles might be needed for robust results, most efficient initial particle distributions, etc.

The transport quantities are at the very end. 2000 particles across the section seems to give OK results but ideally might want more - s.d. in total transport across IFR of maybe 10-20% by selecting a different random 2000 particles. Also this is just a snapshot (though using monthly means). With a few (<10) different crossing times I've seen total transports between 2 and 7 Sv, so with 2000 particles sampling errors probably small compared to variability.

The results for 2016 shown here suggest that of the 6.5 Sv crossing IFR 0.6 Sv came through Rockall Trough, with most of the rest coming north west of Rockall Bank, and about 0.5 Sv where I haven't identified the source. My initial feeling from other years is that most of the variability is in the flow from W of Rockall Bank with always around 0.5 Sv through Rockall Trough.

I haven't looked at Faroe-Shetland Channel yet, so this is only part (and probably the smaller part) of the northward flow in NE Atlantic towards Arctic.

So, here we
- load lagrangian trajectory data that has been written by OceanParcels from particle tracking in Viking20x model (currently using monthly mean fields for speed)
    - particles tracked from a section across IFR
    - random release points along section
    - tracked forward for 6 months to id destinations
    - tracked backwards for 2 years to id sources
- plot sections
- plot trajectories
- subset trajectories based on source and destination
- estimate transports

Transports associated with different pathways are estimated by assuming that each particle crossing IFR has an associated volume transport, which it maintains throughout its life, given by the 
                velocity across the section * total cross-sectional area / number of particles
So like a streamtube...    

## To do

- calculate section area properly 
- explore sensitivity to particle numbers and random seed (i.e. do I have enough particles)
- calculate true transport across section in model
- not selecting ocean particles properly near bed - partial cells
- fix time units on track plots

## Technical preamble

In [ ]:
%matplotlib inline

import matplotlib.colors as colors
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import xarray as xr
from datetime import datetime, timedelta
import seaborn as sns
from matplotlib.colors import ListedColormap



In [ ]:
## Parameters

# Parcels output file
path_parcels_out = Path('C:/Users/sa05af/Desktop/Temp/Parcels/') 
filename = 'Parcels_IFFForwards_1m_June2016_2000.nc'
filenameb = 'Parcels_IFFBackwards_1m_June2016_2000.nc'
# filename = 'Parcels_TITANIC_1k.nc'

# model mask file
data_path = Path("C:/Users/sa05af/Desktop/Temp/Parcels/data/iAtlantic/")
experiment_name = "VIKING20X.L46-KKG36107B"

#velocity conversions on IFR (63N)

lat2metres = 60.0*1852.0
lon2metres = 60.0*1852.0*np.cos(np.radians(63.0))


## Load data

In [ ]:
ds = xr.open_dataset(path_parcels_out / filename)

# display(ds)
# ds.isel(obs=0).z.max()

In [ ]:
# ds.vvel.isel(obs=10)

In [ ]:
mesh_mask_file = data_path / "mask" / experiment_name / "1_mesh_mask.nc"

In [ ]:
mesh_mask = xr.open_dataset(mesh_mask_file)
mesh_mask = mesh_mask.squeeze()
mesh_mask = mesh_mask.set_coords(["nav_lon", "nav_lat", "nav_lev"])

bathy = mesh_mask.mbathy.rename("number of water filled points")

depth = (mesh_mask.e3t_0 * mesh_mask.tmask).sum("z")
# display(mesh_mask)

## Have a quick look

### Release positions of the particles on section from Iceland to Faroes

In [ ]:
plt.figure(figsize = (9,4.5))

plt.scatter(
    ds.isel(obs=0).lon.data.flatten(),
    ds.isel(obs=0).z.data.flatten(),
    20,
    ds.isel(obs=0).vvel.data.flatten()*lon2metres
#     , vmin = 0, vmax = 12.5
#    ,cmap= ListedColormap(sns.color_palette("hsv",20))
#    ,alpha=0.3
)

plt.ylim(700,0)

plt.colorbar(label = "vvel [m/s]")


## Load backward data

In [ ]:
# Parcels output file
# filename = 'Parcels_TITANIC_1k.nc'

dsb = xr.open_dataset(path_parcels_out / filenameb)
# display(dsb)
# print(dsb.time.min())

ds_full = xr.concat([dsb,ds],dim='obs')

# display(ds_full)

### Trajectories of particles

In [ ]:
plt.figure(figsize = (12,6))

# plot tracks as scatter plots

plt.scatter(
    ds_full.where(ds_full.time < np.datetime64('2019-06-15')).lon.data.flatten(),
    ds_full.where(ds_full.time < np.datetime64('2019-06-15')).lat.data.flatten(),
    3,
    ds_full.where(ds_full.time < np.datetime64('2019-06-15')).time.data.flatten()
#     ,alpha=0.3, vmin = 0, vmax = 1000
#    ,alpha=0.3,cmap= ListedColormap(sns.color_palette("BrBG",8))
#    ,alpha=0.3
)
plt.colorbar(label = "time [stupid units. range 2.5 years, all cross IFR at 2 years]");

# plot depth contours and land boundary from model bathymetry
depth.isel(y=slice(1800, 2499), x=slice(1300, 2404)).plot.contour(
    x="nav_lon", y="nav_lat", colors = 'grey', levels = [200,800,1500,2000,2500,3500]
);
depth.isel(y=slice(1800, 2499), x=slice(1300, 2404)).plot.contour(
    x="nav_lon", y="nav_lat", colors = 'k', levels = [1]
);
# plt.savefig('upstream.png')
plt.ylim(55,70)
plt.xlim(-30,10)


### Find subsets of trajectories passing through different areas, either upstream or downstream

This just tests whether particles were ever in an area, much easier than testing line crossings.

In [ ]:
def apply_through_area(ds, min_lon, max_lon, min_lat, max_lat, criterion_name):
    '''Apply an area crossing criterion.
    
    Larvae in ds selected if they pass through given area.
    '''
    # particles are selected if they pass through given area.
    particle_out =  (
                    (ds.lon > min_lon) * 
                    (ds.lon < max_lon) *
                    (ds.lat > min_lat) * 
                    (ds.lat < max_lat)
                    )
                    
#     particle_out = (ds.temp < max_lat)
    

    # last place in array determines if particle went through area  
    is_thru = (particle_out.cumsum("obs") != 0)
    
#     print(particle_out.max())
    
    # add data to original ds
    ds[criterion_name] = is_thru
    
    return ds

In [ ]:
# comes through Rockall Trough
ds_area = apply_through_area(ds_full,-14.0,-9.0,57.25,57.75,'thru_RT')    
# comes northward west of Rockall Bank
ds_area = apply_through_area(ds_area,-60,-14,57.25,57.75,'westof_RB')  
# comes clockwise round Iceland
ds_area = apply_through_area(ds_area,-22,-20,65,70,'round_I')  
# comes from NW
ds_area = apply_through_area(ds_area,-20,-10,68.5,70,'from_NW')  
# goes northeastward towards Arctic - all particles doing this pass north of Faroe
ds_area = apply_through_area(ds_area,-6.5,-5.5,62.3,64.5,'crossed_IFR')    
# goes west south of Iceland in SPG
ds_area = apply_through_area(ds_area,-23,-22,62.,64.,'went_west')         

### Plot tracks for a couple of cases

In [ ]:
plt.figure(figsize = (11,8))

plt.scatter(
    ds_area.where(ds_area.isel(obs=-1).crossed_IFR).where(ds_area.isel(obs=-1).westof_RB).lon.data.flatten(),
    ds_area.where(ds_area.isel(obs=-1).crossed_IFR).where(ds_area.isel(obs=-1).westof_RB).lat.data.flatten(),
    3,
    ds_area.where(ds_area.isel(obs=-1).crossed_IFR).where(ds_area.isel(obs=-1).westof_RB).time.data.flatten()
#     , vmin = 0, vmax = 12.5
#    ,cmap= ListedColormap(sns.color_palette("hsv",20))
#    ,alpha=0.3
)
plt.colorbar(label = "time [stupid units. range 2.5 years, all cross IFR at 2 years]")

# plt.plot(titanic_lon, titanic_lat, 'rx', ms = 8, mew = 2);

depth.isel(y=slice(1800, 2499), x=slice(1300, 2404)).plot.contour(
    x="nav_lon", y="nav_lat", colors = 'grey', levels = [200,800,1500,2000,2500,3500]
);
depth.isel(y=slice(1800, 2499), x=slice(1300, 2404)).plot.contour(
    x="nav_lon", y="nav_lat", colors = 'k', levels = [1]
);
# plt.savefig('upstream.png')
# plt.ylim(45,68)
# plt.xlim(-35,15)
plt.ylim(55,65)
plt.xlim(-25,-2.5)
plt.title('Trajectories from west of Rockall Bank')


In [ ]:
plt.figure(figsize = (11,8))

plt.scatter(
    ds_area.where(ds_area.isel(obs=-1).crossed_IFR).where(ds_area.isel(obs=-1).thru_RT).lon.data.flatten(),
    ds_area.where(ds_area.isel(obs=-1).crossed_IFR).where(ds_area.isel(obs=-1).thru_RT).lat.data.flatten(),
    3,
    ds_area.where(ds_area.isel(obs=-1).crossed_IFR).where(ds_area.isel(obs=-1).thru_RT).time.data.flatten()
#     , vmin = 0, vmax = 12.5
#    ,cmap= ListedColormap(sns.color_palette("hsv",20))
#    ,alpha=0.3
)
plt.colorbar(label = "time [stupid units. range 2.5 years, all cross IFR at 2 years]")

# plt.plot(titanic_lon, titanic_lat, 'rx', ms = 8, mew = 2);

depth.isel(y=slice(1800, 2499), x=slice(1300, 2404)).plot.contour(
    x="nav_lon", y="nav_lat", colors = 'grey', levels = [200,800,1500,2000,2500,3500]
);
depth.isel(y=slice(1800, 2499), x=slice(1300, 2404)).plot.contour(
    x="nav_lon", y="nav_lat", colors = 'k', levels = [1]
);
# plt.savefig('upstream.png')
# plt.ylim(45,68)
# plt.xlim(-35,15)
plt.ylim(55,65)
plt.xlim(-25,-2.5)
plt.title('Trajectories from west of Rockall Bank')


## Plot IFF section coloured by source

In [ ]:
# ds.isel(obs=0).time.plot();
plt.figure(figsize = (9,4.5))

plt.scatter(
    ds.isel(obs=0).lon.data.flatten(),
    ds.isel(obs=0).z.data.flatten(),
    s = 20, color = 'k', label='from other, went other'
)
plt.scatter(
    ds.where(ds_area.isel(obs=-1).crossed_IFR).isel(obs=0).lon.data.flatten(),
    ds.where(ds_area.isel(obs=-1).crossed_IFR).isel(obs=0).z.data.flatten(),
    s = 20, color = 'b', label='from other, went NE'
)
plt.scatter(
    ds.where(ds_area.isel(obs=-1).went_west).isel(obs=0).lon.data.flatten(),
    ds.where(ds_area.isel(obs=-1).went_west).isel(obs=0).z.data.flatten(),
    s = 20, color = 'coral', label='from other, went west'
)
plt.scatter(
    ds.where(ds_area.isel(obs=-1).westof_RB).isel(obs=0).lon.data.flatten(),
    ds.where(ds_area.isel(obs=-1).westof_RB).isel(obs=0).z.data.flatten(),
    s = 20, color = 'g', label='from WRB, went other'
)
plt.scatter(
    ds.where(ds_area.isel(obs=-1).crossed_IFR).where(ds_area.isel(obs=-1).westof_RB).isel(obs=0).lon.data.flatten(),
    ds.where(ds_area.isel(obs=-1).crossed_IFR).where(ds_area.isel(obs=-1).westof_RB).isel(obs=0).z.data.flatten(),
    s = 20, color = 'lightblue', label='from WRB, went NE'
)
plt.scatter(
    ds.where(ds_area.isel(obs=-1).went_west).where(ds_area.isel(obs=-1).westof_RB).isel(obs=0).lon.data.flatten(),
    ds.where(ds_area.isel(obs=-1).went_west).where(ds_area.isel(obs=-1).westof_RB).isel(obs=0).z.data.flatten(),
    s = 20, color = 'pink', label='from WRB, went west'
)
plt.scatter(
    ds.where(ds_area.isel(obs=-1).thru_RT).isel(obs=0).lon.data.flatten(),
    ds.where(ds_area.isel(obs=-1).thru_RT).isel(obs=0).z.data.flatten(),
    s = 20, color = 'r', label='from RT, went other'
)
plt.scatter(
    ds.where(ds_area.isel(obs=-1).crossed_IFR).where(ds_area.isel(obs=-1).thru_RT).isel(obs=0).lon.data.flatten(),
    ds.where(ds_area.isel(obs=-1).crossed_IFR).where(ds_area.isel(obs=-1).thru_RT).isel(obs=0).z.data.flatten(),
    s = 20, color = 'yellow', label='from RT, went NE'
)
plt.scatter(
    ds.where(ds_area.isel(obs=-1).went_west).where(ds_area.isel(obs=-1).thru_RT).isel(obs=0).lon.data.flatten(),
    ds.where(ds_area.isel(obs=-1).went_west).where(ds_area.isel(obs=-1).thru_RT).isel(obs=0).z.data.flatten(),
    s = 20, color = 'lightgreen', label='from RT, went west'
)

plt.legend(ncol=3)

plt.ylim(800,0)

## Estimate transports across IFR

In [ ]:
sectionAreaEW = 500.0 * 237000.0  # very approx. mean depth x length between slopes.
sectionAreaNS = 500.0 * 209000.0

nParticles = 963.0

NEwardsParticleTransport = (sectionAreaEW*dsb.isel(obs=0).vvel*lat2metres +
                            sectionAreaNS*dsb.isel(obs=0).uvel*lon2metres)/nParticles


print(filename)
print('Total transport across IFR.............................',
      NEwardsParticleTransport.sum().data)
print('...from west of Rockall Bank...........................',
      NEwardsParticleTransport.where(ds_area.isel(obs=-1).westof_RB).sum().data)
print('.........from Rockall Trough...........................',
      NEwardsParticleTransport.where(ds_area.isel(obs=-1).thru_RT).sum().data)
print('.........proceeds northwards...........................',
      NEwardsParticleTransport.where(ds_area.isel(obs=-1).crossed_IFR).sum().data)
print('...from west of Rockall Bank....and proceeds northwards',
      NEwardsParticleTransport.where(ds_area.isel(obs=-1).crossed_IFR).where(ds_area.isel(obs=-1).westof_RB).sum().data)
print('.........from Rockall Trough....and proceeds northwards',
      NEwardsParticleTransport.where(ds_area.isel(obs=-1).crossed_IFR).where(ds_area.isel(obs=-1).thru_RT).sum().data)






In [ ]:
plt.figure(figsize = (11,8))

plt.scatter(
    ds_area.where(ds_area.isel(obs=-1).round_I).where(ds_area.isel(obs=-1).went_west).lon.data.flatten(),
    ds_area.where(ds_area.isel(obs=-1).round_I).where(ds_area.isel(obs=-1).went_west).lat.data.flatten(),
    3,
    ds_area.where(ds_area.isel(obs=-1).round_I).where(ds_area.isel(obs=-1).went_west).time.data.flatten()
#     , vmin = 0, vmax = 12.5
#    ,cmap= ListedColormap(sns.color_palette("hsv",20))
#    ,alpha=0.3
)
plt.colorbar(label = "time [stupid units. range 2.5 years, all cross IFR at 2 years]")

# plt.plot(titanic_lon, titanic_lat, 'rx', ms = 8, mew = 2);

depth.isel(y=slice(1800, 2499), x=slice(1300, 2404)).plot.contour(
    x="nav_lon", y="nav_lat", colors = 'grey', levels = [200,800,1500,2000,2500,3500]
);
depth.isel(y=slice(1800, 2499), x=slice(1300, 2404)).plot.contour(
    x="nav_lon", y="nav_lat", colors = 'k', levels = [1]
);
# plt.savefig('upstream.png')
# plt.ylim(45,68)
# plt.xlim(-35,15)
plt.ylim(47.5,70)
plt.xlim(-30,10)
plt.title('Trajectories from Rockall Trough')
